# 2.2 ClimateWins Keras- Hyperparameter Optimization with Bayesian Search

## Contents:

### 1. Importing Libraries and Data
- Libraries in line with Keras, 2.2 scripts
- Data: unscaled (observations), pleasant weather (predictions)

### 2. Data Wranging

### 3. Reshaping for ML modeling and Bayesian Optomization
the Bayesian optimization requires a slightly different form for your X and y sets. Before you split them:

-Your X shape will need to be in the form (22950, 15, 9);

-Your y shape will need to be in the form (22950,);

You can use the argmax function shown in the Exercise to transform your y set.

### 4. Split data

## 5. Bayesian Hyperparameter Optimzation

### 6. Create Keras Model


- Compile and Run the model

- Create confusion matrix of results
    

In [1]:
# Importing libraries for Keras, CNNs, and Hyperparameter Optimization

import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import train_test_split, cross_val_score
import tensorflow as tf
from sklearn import datasets  
from sklearn.ensemble import RandomForestClassifier
from numpy import argmax
from sklearn import metrics  
from sklearn.tree import plot_tree
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [2]:
print (keras.__version__)

3.3.3


In [3]:
# Setting Path 

path = r'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Specialization - Machine Learning/Real World Applications of Machine Learning/ML Achievement 2 MASTER FOLDER'
path

'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Specialization - Machine Learning/Real World Applications of Machine Learning/ML Achievement 2 MASTER FOLDER'

In [4]:
# delimiting columns displayed
pd.options.display.max_columns = None

In [5]:
# Importing prediction data

pleasant = pd.read_csv (os.path.join(path, '02 Data', 'pleasant_wx_predictions.csv'), index_col = False)

In [6]:
# Importing unscaled observation data

unscaled =  pd.read_csv (os.path.join(path, '02 Data', 'wx_unscaled.csv'), index_col = False)

In [7]:
unscaled.shape

(22950, 170)

In [8]:
pleasant.shape

(22950, 16)

### 2. Data Wrangling

- Ensure wx data is structured with correct shape to feed the deep learning model.
    - drop DATE and MONTH from observations, DATE from predictions
    - drop 3 wx stations not included in pleasant_wx
    - 2 types of observations (columns) are missing multiple years for most wx stations; remove them
    - 3 individual observations need to be filled in
    - Export data as "Cleaned" version, X shape should be (22950, 135) and y shape should be (22950, 15)

In [9]:
# dropping all columns regarding Gdansk, Roma, Tours from unscaled, as they aren't included in pleasant_wx


columns_to_drop = ['GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation', 
                   'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 
                   'GDANSK_temp_max', 'ROMA_cloud_cover', 'ROMA_wind_speed', 
                   'ROMA_humidity', 'ROMA_pressure', 'ROMA_sunshine', 'ROMA_temp_mean',
                   'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure',
                   'TOURS_global_radiation', 'TOURS_precipitation', 'TOURS_temp_mean', 
                   'TOURS_temp_min', 'TOURS_temp_max']

# Using the drop() method to drop the specified columns
unscaled.drop(columns=columns_to_drop, inplace=True)

In [10]:
unscaled.shape 

(22950, 149)

In [11]:
unscaled.drop(columns = ['DATE', 'MONTH'], inplace = True)

In [12]:
pleasant.drop(columns = 'DATE', inplace = True) 

In [13]:
pleasant.shape

# prediction dataset is correct shape

(22950, 15)

Two types of observations are missing multiple years for most weather stations. Remove them.

- wind speed (only 11 stations)
- snow depth (only 7 stations)

In [14]:
further_drops = ['BASEL_snow_depth',  'DUSSELDORF_snow_depth', 'HEATHROW_snow_depth',
                  'MUNCHENB_snow_depth', 'OSLO_snow_depth',  'VALENTIA_snow_depth',
                 'BASEL_wind_speed', 'DEBILT_wind_speed', 'DUSSELDORF_wind_speed',
                  'KASSEL_wind_speed', 'LJUBLJANA_wind_speed',  'MAASTRICHT_wind_speed',
                  'MADRID_wind_speed', 'OSLO_wind_speed','SONNBLICK_wind_speed',]

unscaled.drop(columns=further_drops, inplace=True)

In [15]:
# Inserting "dummy values" for the 3 that are still missing

# missing 'Kassel_cloud_cover', with 'Ljubjana_cloud_cover'
# missing 'Munchenb_pressure' with 'Sonnblick_pressure'
# missing 'Stockholm_humidity' with Oslo_humidity'

unscaled.columns.get_loc('HEATHROW_temp_max')

53

In [16]:
unscaled.columns.get_loc('MUNCHENB_humidity')

90

In [17]:
unscaled.columns.get_loc('STOCKHOLM_cloud_cover')

115

In [18]:
unscaled.insert(54,'KASSEL_cloud_cover', unscaled['LJUBLJANA_cloud_cover'])
unscaled.insert(92,'MUNCHENB_pressure',unscaled['SONNBLICK_pressure'])
unscaled.insert(118, 'STOCKHOLM_humidity', unscaled['OSLO_humidity'])
unscaled.columns.tolist()

['BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSSELDORF_temp_mean',


In [19]:
unscaled.shape

(22950, 135)

In [20]:
unscaled.to_pickle(os.path.join(path, '02 Data', 'X_cleaned.pkl'))

### 3. Reshaping for ML modeling
- Ensure the layers can be fed to the deep learning model correctly. 
- You’ll need to split the observations (X) into 15 groups of 9 types of observations, and your labels (y) should also be in 15 groups 
    - (it doesn’t need to be transformed or reshaped). 
        - The final shapes should be X = (22950, 15, 9) and y = (22950, 15).


In [21]:
X = pd.read_pickle(os.path.join(path, '02 Data', 'X_cleaned.pkl'))

In [22]:
y = pleasant

In [23]:
X.shape

(22950, 135)

In [24]:
# turning X and y from a df to arrays

X = np.array(X)
y = np.array(y)

In [25]:
X = X.reshape(-1,15,9)

In [26]:
y =  np.argmax(y, axis = 1)

In [27]:
# verifying array shape

y

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

### 4. Splitting data (training and test sets)

In [29]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 39)

## 5. Bayesian hyperparameter optimization

In [35]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from sklearn.model_selection import StratifiedKFold, cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np

# Ensure global variables are defined
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(np.unique(y_train))  # or another method to count classes

# Define score function
score_acc = 'accuracy'

In [30]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 #_count_classes(Y_train) #Number of weather stations!!
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [36]:
# Create function
def bay_area_rnn(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
                 layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU, 'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)  # This won't be used in RNN but kept for consistency
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)] #optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def rnn_model():
        model = Sequential()
        model.add(LSTM(neurons, input_shape=(timesteps, input_dim), activation=activation))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax'))  # Don't use relu here
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model

    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=rnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score


In [37]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), #(10, 50), #
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area_rnn, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32


/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 - 1s - 82ms/step - accuracy: 0.5990 - loss: 2.6373
Epoch 2/32
15/15 - 0s - 29ms/step - accuracy: 0.6425 - loss: 2.6315
Epoch 3/32
15/15 - 0s - 29ms/step - accuracy: 0.6425 - loss: 2.6282
Epoch 4/32
15/15 - 0s - 29ms/step - accuracy: 0.6425 - loss: 2.6254
Epoch 5/32
15/15 - 0s - 30ms/step - accuracy: 0.6425 - loss: 2.6229
Epoch 6/32
15/15 - 0s - 30ms/step - accuracy: 0.6425 - loss: 2.6207
Epoch 7/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6186
Epoch 8/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6167
Epoch 9/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6149
Epoch 10/32
15/15 - 1s - 34ms/step - accuracy: 0.6425 - loss: 2.6132
Epoch 11/32
15/15 - 0s - 33ms/step - accuracy: 0.6425 - loss: 2.6115
Epoch 12/32
15/15 - 0s - 33ms/step - accuracy: 0.6425 - loss: 2.6100
Epoch 13/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6085
Epoch 14/32
15/15 - 0s - 33ms/step - accuracy: 0.6425 - loss: 2.6071
Epoch 15/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 - 1s - 79ms/step - accuracy: 0.6063 - loss: 2.6356
Epoch 2/32
15/15 - 0s - 30ms/step - accuracy: 0.6425 - loss: 2.6315
Epoch 3/32
15/15 - 0s - 30ms/step - accuracy: 0.6425 - loss: 2.6281
Epoch 4/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6253
Epoch 5/32
15/15 - 0s - 30ms/step - accuracy: 0.6425 - loss: 2.6229
Epoch 6/32
15/15 - 0s - 30ms/step - accuracy: 0.6425 - loss: 2.6206
Epoch 7/32
15/15 - 0s - 31ms/step - accuracy: 0.6425 - loss: 2.6186
Epoch 8/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6167
Epoch 9/32
15/15 - 0s - 33ms/step - accuracy: 0.6425 - loss: 2.6149
Epoch 10/32
15/15 - 1s - 35ms/step - accuracy: 0.6425 - loss: 2.6131
Epoch 11/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6115
Epoch 12/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6100
Epoch 13/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6085
Epoch 14/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 - loss: 2.6070
Epoch 15/32
15/15 - 0s - 32ms/step - accuracy: 0.6425 

...

21/21 - 2s - 102ms/step - accuracy: 0.3446 - loss: 1.9945
Epoch 2/64
21/21 - 0s - 14ms/step - accuracy: 0.6495 - loss: 1.0292
Epoch 3/64
21/21 - 0s - 14ms/step - accuracy: 0.6763 - loss: 0.9255
Epoch 4/64
21/21 - 0s - 14ms/step - accuracy: 0.6936 - loss: 0.8807
Epoch 5/64
21/21 - 0s - 15ms/step - accuracy: 0.7120 - loss: 0.8436
Epoch 6/64
21/21 - 0s - 16ms/step - accuracy: 0.7224 - loss: 0.8154
Epoch 7/64
21/21 - 0s - 16ms/step - accuracy: 0.7363 - loss: 0.7901
Epoch 8/64
21/21 - 0s - 16ms/step - accuracy: 0.7445 - loss: 0.7622
Epoch 9/64
21/21 - 0s - 14ms/step - accuracy: 0.7559 - loss: 0.7348
Epoch 10/64
21/21 - 0s - 16ms/step - accuracy: 0.7602 - loss: 0.7214
Epoch 11/64
21/21 - 0s - 16ms/step - accuracy: 0.7715 - loss: 0.6957
Epoch 12/64
21/21 - 0s - 16ms/step - accuracy: 0.7805 - loss: 0.6675
Epoch 13/64
21/21 - 0s - 16ms/step - accuracy: 0.7832 - loss: 0.6554
Epoch 14/64
21/21 - 0s - 16ms/step - accuracy: 0.7911 - loss: 0.6354
Epoch 15/64
21/21 - 0s - 16ms/step - accuracy: 0.7950

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 - 1s - 51ms/step - accuracy: 0.4943 - loss: 1.6725
Epoch 2/64
21/21 - 0s - 14ms/step - accuracy: 0.6391 - loss: 1.0496
Epoch 3/64
21/21 - 0s - 15ms/step - accuracy: 0.6553 - loss: 0.9407
Epoch 4/64
21/21 - 0s - 14ms/step - accuracy: 0.6843 - loss: 0.8766
Epoch 5/64
21/21 - 0s - 16ms/step - accuracy: 0.7102 - loss: 0.8282
Epoch 6/64
21/21 - 0s - 16ms/step - accuracy: 0.7275 - loss: 0.7870
Epoch 7/64
21/21 - 0s - 16ms/step - accuracy: 0.7394 - loss: 0.7568
Epoch 8/64
21/21 - 0s - 16ms/step - accuracy: 0.7520 - loss: 0.7291
Epoch 9/64
21/21 - 0s - 16ms/step - accuracy: 0.7633 - loss: 0.7006
Epoch 10/64
21/21 - 0s - 16ms/step - accuracy: 0.7660 - loss: 0.6845
Epoch 11/64
21/21 - 0s - 16ms/step - accuracy: 0.7737 - loss: 0.6679
Epoch 12/64
21/21 - 0s - 16ms/step - accuracy: 0.7805 - loss: 0.6544
Epoch 13/64
21/21 - 0s - 17ms/step - accuracy: 0.7898 - loss: 0.6323
Epoch 14/64
21/21 - 0s - 16ms/step - accuracy: 0.7904 - loss: 0.6240
Epoch 15/64
21/21 - 0s - 16ms/step - accuracy: 0.7924 

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 - 1s - 52ms/step - accuracy: 0.4748 - loss: 1.6603
Epoch 2/64
21/21 - 0s - 14ms/step - accuracy: 0.6596 - loss: 1.0183
Epoch 3/64
21/21 - 0s - 14ms/step - accuracy: 0.6731 - loss: 0.9337
Epoch 4/64
21/21 - 0s - 14ms/step - accuracy: 0.6857 - loss: 0.8931
Epoch 5/64
21/21 - 0s - 15ms/step - accuracy: 0.6943 - loss: 0.8652
Epoch 6/64
21/21 - 0s - 16ms/step - accuracy: 0.7146 - loss: 0.8195
Epoch 7/64
21/21 - 0s - 16ms/step - accuracy: 0.7322 - loss: 0.7766
Epoch 8/64
21/21 - 0s - 16ms/step - accuracy: 0.7525 - loss: 0.7400
Epoch 9/64
21/21 - 0s - 16ms/step - accuracy: 0.7572 - loss: 0.7215
Epoch 10/64
21/21 - 0s - 16ms/step - accuracy: 0.7678 - loss: 0.6960
Epoch 11/64
21/21 - 0s - 16ms/step - accuracy: 0.7738 - loss: 0.6729
Epoch 12/64
21/21 - 0s - 16ms/step - accuracy: 0.7805 - loss: 0.6534
Epoch 13/64
21/21 - 0s - 16ms/step - accuracy: 0.7849 - loss: 0.6471
Epoch 14/64
21/21 - 0s - 16ms/step - accuracy: 0.7850 - loss: 0.6390
Epoch 15/64
21/21 - 0s - 16ms/step - accuracy: 0.7915 

In [38]:
best_params = nn_opt.max['params']
best_score = nn_opt.max['target']

print(f"Best Parameters: {best_params}")
print(f"Highest Accuracy: {best_score}")

Best Parameters: {'activation': 2.0021111271343606, 'batch_size': 669.5648845624361, 'dropout': 0.8046122554274766, 'dropout_rate': 0.04147123592409228, 'epochs': 64.3682542554042, 'kernel': 1.6309861266954004, 'layers1': 2.3041514571471446, 'layers2': 1.279715127746877, 'learning_rate': 0.7175216868627151, 'neurons': 37.74727789303107, 'normalization': 0.8666093251432194, 'optimizer': 1.2737223001049398}
Highest Accuracy: 0.835870336583014


In [39]:
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'LeakyReLU', 'relu']
optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']

# Best parameters from optimization
activation_index = round(best_params['activation'])
optimizer_index = round(best_params['optimizer'])

best_params['activation'] = activationL[activation_index]
best_params['optimizer'] = optimizerL[optimizer_index]
best_params['neurons'] = round(best_params['neurons'])
best_params['kernel'] = round(best_params['kernel'])
best_params['batch_size'] = round(best_params['batch_size'])
best_params['epochs'] = round(best_params['epochs'])
best_params['layers1'] = round(best_params['layers1'])
best_params['layers2'] = round(best_params['layers2'])

print(f"Best Parameters (interpreted): {best_params}")


Best Parameters (interpreted): {'activation': 'softplus', 'batch_size': 670, 'dropout': 0.8046122554274766, 'dropout_rate': 0.04147123592409228, 'epochs': 64, 'kernel': 2, 'layers1': 2, 'layers2': 1, 'learning_rate': 0.7175216868627151, 'neurons': 38, 'normalization': 0.8666093251432194, 'optimizer': 'Adam'}


### 6. Keras Model

In [45]:
# Check unique labels in your training and validation data
unique_labels_train = np.unique(y_train)
unique_labels_test = np.unique(y_test)

print("Unique labels in training data:", unique_labels_train)
print("Unique labels in validation data:", unique_labels_val)

# Ensure all labels are in the range [0, 13]
y_train_fixed = np.where(y_train >= 14, 13, y_train)
y_test_fixed = np.where(y_test >= 14, 13, y_test)

unique_labels_train_fixed = np.unique(y_train_fixed)
unique_labels_test_fixed = np.unique(y_test_fixed)

print("Fixed unique labels in training data:", unique_labels_train_fixed)
print("Fixed unique labels in validation data:", unique_labels_test_fixed)

# Now use y_train_fixed and y_val_fixed for training
model.fit(X_train, y_train_fixed, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2, validation_data=(X_test, y_test_fixed))


Unique labels in training data: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
Unique labels in validation data: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
Fixed unique labels in training data: [ 0  1  2  3  4  5  6  7  8  9 10 11 13]
Fixed unique labels in validation data: [ 0  1  2  3  4  5  6  7  8  9 10 11 13]
Epoch 1/64
26/26 - 0s - 18ms/step - accuracy: 0.5718 - loss: 1.5892 - val_accuracy: 0.5891 - val_loss: 1.2953
Epoch 2/64
26/26 - 0s - 15ms/step - accuracy: 0.6334 - loss: 1.2010 - val_accuracy: 0.6460 - val_loss: 1.1466
Epoch 3/64
26/26 - 0s - 15ms/step - accuracy: 0.6425 - loss: 1.1229 - val_accuracy: 0.6460 - val_loss: 1.1198
Epoch 4/64
26/26 - 0s - 15ms/step - accuracy: 0.6425 - loss: 1.0988 - val_accuracy: 0.6460 - val_loss: 1.1167
Epoch 5/64
26/26 - 0s - 18ms/step - accuracy: 0.6425 - loss: 1.0798 - val_accuracy: 0.6460 - val_loss: 1.1078
Epoch 6/64
26/26 - 0s - 16ms/step - accuracy: 0.6425 - loss: 1.0679 - val_accuracy: 0.6460 - val_loss: 1.0996
Epoch 7/64
26/26 - 0s - 16

In [57]:
# Assuming the labels should be in the range of 0 to 13
y_train_corrected = np.where(y_train == 14, 13, y_train)  # Replace 14 with 13
unique_labels_corrected = np.unique(y_train_corrected)
print("Corrected unique labels in y_train:", unique_labels_corrected)


Corrected unique labels in y_train: [ 0  1  2  3  4  5  6  7  8  9 10 11 13]


In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl

# Best parameters from optimization
best_params = {
    'neurons': 38,
    'kernel': 2,
    'activation': 'softplus',
    'optimizer': 'Adam',
    'learning_rate': 0.7175216868627151,
    'batch_size': 670,
    'epochs': 64,
    'layers1': 2,
    'layers2': 1,
    'normalization': 0.8666093251432194,
    'dropout': 0.8046122554274766,
    'dropout_rate':  0.04147123592409228,
}

# Initialize optimizer with learning rate
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}

optimizer = optimizers[best_params['optimizer']]

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(set(y_train))  # Ensure n_classes is correctly calculated

model = Sequential()
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation']))

if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate']))

for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))  # Ensure n_classes is correctly calculated

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with the optimized parameters
model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2)


Epoch 1/64


2024-07-03 09:22:24.065794: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 14 which is outside the valid range of [0, 14).  Label values: 0 0 1 1 0 0 0 0 0 0 0 2 0 9 0 1 3 0 0 1 9 0 1 0 0 9 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 5 0 0 2 2 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 4 9 0 0 1 0 9 0 1 1 1 0 0 0 0 0 0 0 1 0 9 0 0 0 5 0 1 0 0 0 0 0 0 1 0 3 0 1 7 1 0 0 0 0 5 0 0 0 0 0 9 0 0 0 1 0 0 0 1 0 0 0 0 0 0 9 0 1 0 0 0 0 0 0 0 9 2 1 0 0 9 1 0 0 0 1 0 0 4 5 1 0 2 9 1 9 0 0 9 0 0 0 1 0 0 0 1 0 1 2 1 1 0 0 0 3 0 1 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 1 1 0 0 2 0 0 0 3 1 1 1 9 0 1 2 0 9 0 0 0 0 8 0 0 0 0 9 1 3 1 0 0 0 0 0 0 0 9 0 9 0 0 1 0 0 1 0 1 1 0 9 1 0 0 0 1 1 0 0 0 0 0 0 9 0 0 0 1 1 0 0 0 1 6 1 0 0 0 1 0 0 2 1 0 0 0 1 1 0 0 0 0 0 1 14 0 0 1 0 1 0 1 0 0 9 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 5 1 1 5 1 0 0 1 0 0 0 0 0 1 0 1 0 0 3 0 0 6 1 0 1 1 0 0 0 1 5 0 0 1 9 1 9 0 0 0 0 0 0 0 1 0 0 0 7 9 5 0

InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/jeremyobach/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Users/jeremyobach/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  F

# Best parameters from optimization
best_params = {
    'neurons': 38,
    'kernel': 2,
    'activation': 'softplus',
    'optimizer': 'Adam',
    'learning_rate': 0.7175216868627151,
    'batch_size': 670,
    'epochs': 64,
    'layers1': 2,
    'layers2': 1,
    'normalization': 0.8666093251432194,
    'dropout': 0.8046122554274766,
    'dropout_rate':  0.04147123592409228,
}

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl

# Initialize optimizer with learning rate
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}

optimizer = optimizers[best_params['optimizer']]

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
#n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation'], input_shape=(15,9)))

if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate'], seed=123))

for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))
  

model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softplus')) # sigmoid, tanh, softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with the optimized parameters
model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2)


In [47]:
model.summary()

Model: "sequential_102"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 38)         │           722 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 14, 38)         │           152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_483 (Dense)               │ (None, 14, 38)         │         1,482 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_484 (Dense)               │ (None, 14, 38)         │         1,482 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_57 (Dropout)            │ (None, 14, 38)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_485 (Dense)               │ (None, 14, 38)         │         1,482 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 38)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_97 (Flatten)            │ (None, 266)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_486 (Dense)               │ (None, 14)             │         3,738 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,024 (105.57 KB)

 Trainable params: 8,982 (35.09 KB)

 Non-trainable params: 76 (304.00 B)

 Optimizer params: 17,966 (70.18 KB)

In [48]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [49]:
from tensorflow.keras.utils import to_categorical


# Assuming X_train and y_train are your input data and labels
# One-hot encode y_train
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [50]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [56]:
model.fit(X_train, y_train_one_hot, batch_size=460, epochs=91, verbose=2)

Epoch 1/91


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 15), output.shape=(None, 14)

In [52]:
#Change this to Weather true/false
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [53]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])



In [54]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step


In [55]:
cm = confusion_matrix(y_test, y_pred, stations)
print(cm)

Pred        BELGRADE  DEBILT
True                        
BASEL            751    2956
BELGRADE         434     647
BUDAPEST          54     142
DEBILT            14      75
DUSSELDORF        10      23
HEATHROW           7     100
KASSEL             3      12
LJUBLJANA         12      57
MAASTRICHT         2       8
MADRID            40     369
MUNCHENB           4       6
OSLO               2       5
STOCKHOLM          0       1
VALENTIA           0       4
